<a href="https://colab.research.google.com/github/ufrpe-ensino/curso-mineracao-textos/blob/master/01_PLN_spaCy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# spaCy

spaCy é uma biblioteca para PLN que se tornado bastante popular nos últimos anos. Ela é mais recente que o NLTK, projetada especificamente para i) trabalhar em problemas maiores e ii) ocultar detalhes irrelevantes para o  usuário. Vamos nos concentrar nos principais recursos:

[Carregando modelos pré-treinados](#loading)<br>

[Tokenization](#tokenization)<br>

[Lemmatization](#lemma)<br>

[Named entity recognition (NER)](#ner)<br>

[Vizualizando NER](#visualize-ner)<br>

[Word vectors e Similaridades](#vectors)<br>


# Instalação

In [ ]:
!pip install spacy
!spacy download pt

In [ ]:
import os
import spacy
import numpy as np
import pandas as pd

## Loading spaCy models <a id='loading'></a>

O spaCy disponibiliza uma série de [modelos](https://spacy.io/usage/models) pré-treinados, que devem ser baixados e podem funcionar em problemas mais genéricos. spaCy possui modelos diferentes para diferentes idiomas, inclusive em português. 

Para fazer uso dos modelos, primeiro os carregamos no spaCy com 

`nlp = spacy.load ('en')`, que armazena o modelo em uma variável chamada nlp para nós. O `'en'` significa inglês. Se você deseja processar dados nesses idiomas, primeiro precisa baixar os modelos relevantes e carregá-los de maneira semelhante.

O modelo aqui referenciado para o português `pt` corresponde a um modelo de CNN treinado em um subconjunto da base WikiNER. Além do `pt_core_news_sm`, o spacy possui o `pt_core_news_md` e o `pt_core_news_lg`. Detalhes das bases utilizadas e acurácia de cada modelo podem ser vistos [aqui](https://spacy.io/models/pt). 

In [ ]:
nlp = spacy.load('pt')

O modelo agora pode ser acessado através da variável `nlp`:

In [ ]:
doc = nlp("Eu gostaria que as aboboras viessem com mais sementes.")

## Tokenization <a id='tokenization'></a>

A tokenização no spaCy é bem simples de ser utilizada com os modelos pré-treinados. Quando iteramos sobre um objeto `Doc`, spaCy assume que queremos iterar sobre os tokens.

In [ ]:
for token in doc:
    print(token)

In [ ]:
# quantidade de tokens
len(doc)

Cada `token` em `doc` é uma instância da classe `Token`. Este objeto armazena uma série de informações relevantes. como a representação em string do token (`.text`) o índice (`.idx`), ou ainda aspectos linguísticos como `.is_stop`,  `.is_space`, `.lemma_`  e `.pos_`.

In [ ]:
for token in doc:
    print("{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t{6}\t{7}".format(
        token.text,
        token.idx,
        token.lemma_,
        token.is_stop,
        token.is_punct,
        token.is_space,
        token.pos_,
        token.dep_,
    ))

### Exercício

Verifique a quantidade de tokens considerados stopwords e pontuação do texto abaixo. Imprima uma versão do texto removendo este tipo de tokens

In [ ]:
# Texto
review = '''Este é sem dúvida o pior filme que eu já vi. E acredite em mim, eu vi muitos filmes. A reviravolta inacreditável que o filme faz - passando de um extremamente mau filme "Formas de vida alienígenas habitam a terra", com um filme que tenta espalhar um arquicristiano "O dia do julgamento está próximo, buscar Jesus ou queimar por toda a eternidade em as dívidas ardentes do inferno "mensagem - deixou-me atordoado depois de ter sido atormentado por 85 minutos. Até mesmo os cristãos religiosos devem se envergonhar ou ficar furiosos ao ver suas crenças postadas dessa maneira. Eu não sabia o que fazer comigo quando assisti a atuação horrível que poderia ter sido realizada por crianças de 7 anos de idade. Simplesmente repugnante. Eu não sou cristão nem muito religioso. Mas se eu estivesse, não teria mais medo do Inferno. Rich Christiano mostrou ser algo muito pior.'''

# sua resposta

# Detecção de sentenças

As sentenças detectadas pelo modelo ficam armazenadas no atributo `.sents` do objeto `Doc`:

In [ ]:
doc = nlp("Eu estarei em Recife próxima semana. Será que levo roupa para frio?")
for sent in doc.sents:
    print(sent)

### Exercício

1. Quantas sentenças existem no texto armazenado na variável `review`?

2. Quais são essas sentencas?

In [ ]:
# sua resposta

## POS tagging

Como vimos acima, ao submeter uma string ao modelo, o spacy aplica todo o pipeline de PLN ao texto, incluindo o processo de POS Tagging:

In [ ]:
doc = nlp("Eu estarei em Recife próxima semana. Será que levo roupa para frio?")
for token in doc:
    print(token.text, token.pos_)

Esses rótulos podem ser um pouco difíceis de interpretar... O que significa `PROPN`? e `ADP`? Use a função `explain` para obter as respostas:

In [ ]:
print(spacy.explain('PROPN'))
print(spacy.explain('ADP'))

Observe que o texto (label) é sempre armazenado em um atributo com `_` no fim, pois o spaCy armazena internamente tudo na forma de hashes, para tornar o código mais eficiente:

In [ ]:
for token in doc[:10]:
    print(token.text, token.pos)

### Exercício

Retorne a lista de todas as POS Tags da variável `review` como uma lista de tuplas (word, pos) para cada token no texto. Quais as tags mais frequentes?

Dica: você pode utilizar um `pandas.DataFrame` para facilitar as contagens

In [ ]:
# sua resposta

## Lemmatization <a id='lemma'></a>

O spaCy não disponibiliza muitos detalhes de escolha de algoritmos para Lemmatization, o que funciona bem para a maioria dos casos, diferentemente do NLTK:

In [ ]:
for token in doc:
    print(token.text, token.lemma_)

## Named entity recognition (NER) <a id='ner'></a>

Named entity recognition (NER) é uma das principais tarefas em projetos de recuperação e extração de informação em textos. Muitas tarefas se iniciam a partir da detecção de entidades nomeadas, como a extração de relações por exemplo. 

Em NER, as diferentes entidades nomeadas extraídas são agrupadas por tipo. Por exemplo, "pessoa", "organização", "local", "país" etc. No spaCy, existem muitos [tipos diferentes](https://spacy.io/api/annotation#named-entities) de entidades nomeadas que ele pode extrair com modelos pré-treinados.

As entidades nomeadas no spaCy estão disponíveis como propriedade `ents` de um` Doc`. O `.label_` nos diz o tipo de entidade nomeada.

In [ ]:
doc = nlp("Eu estarei em Recife próxima semana. Será que levo roupa para frio?")
for ent in doc.ents:
    print(ent.text,ent.label_)

In [ ]:
doc = nlp("Geraldo Júlio é o prefeito de Recife.")
for ent in doc.ents:
    print(ent.text,ent.label_)

In [ ]:
#explain também funciona para entidades
spacy.explain('LOC')

### Exercício 1

Extraia todas as entidades nomeadas da variável `review`.

In [ ]:
doc = nlp(review)
for ent in doc.ents:
    print(ent.text,ent.label_)

### Exercício 2
Utilizar a lib do python `wikipedia`, para baixar o conteúdo da página referente ao presidente Bolsonaro no wikipedia em português, e analise quais as entidades presentes nas 10 primeiras sentenças do texto.

In [ ]:
!pip install wikipedia

In [ ]:
import wikipedia
wikipedia.set_lang("pt")
p = wikipedia.page("Jair Bolsonaro")

print(p.url)
print(p.title)

content = p.content 

In [ ]:
# sua resposta

## Visualização NER <a id='visualize-ner'></a>

O displaCy é uma extensão do spaCy para visualização do processo de PLN. 

Após importar a lib `displacy`, podemos usar o método `render` sobre o `doc` criado.

In [ ]:
from spacy import displacy

In [ ]:
displacy.render(doc, style='ent', jupyter=True)

### Exercício

Utilize o `displacy` para visualizar as entidades encontradas em algumas das sentenças do conteúdo da página do wikipedia analisada no exercício anterior.

In [ ]:
# sua resposta

## Parsing de Dependencia

A análise de dependência refere-se a desenhar os relacionamentos entre palavras individuais em uma frase. Assim como o NER, esse é um tópico bem importante em PLN. 


In [ ]:
for token in doc[:18]:
    print(token.text, token.dep_, token.head)

Utilizando o `displacy`:

In [ ]:
displacy.render(doc, style='dep', jupyter=True)

In [ ]:
# modificando a visualização
displacy.render(doc, style='dep', jupyter=True, options={"compact": True})

### Exercício

Crie uma lista para cada tipo de entidade no texto da wikipedia analisado, ou seja: 
- texto (string)
- pos
- lemma
- se é uma stopword (`.is_stop`)
- se é pontuação (`.is_punct`)
- se é um número (`.like_num`)
- a relação de dependência (`.dep_`)

Utilize as listas criadas para criar um `pandas.DataFrame`, e analise a distribuição das categorias registradas.

In [ ]:
doc = nlp(content)
tokens = [token.text for token in doc]

# resposta


## Word vectors e similaridade <a id='vectors'></a>

Os modelos do spaCy em sua versão reduzida (`sm`), não incluem word vectors completos, de modo que é possível obter uma aproximação dessas distâncias. Caso se queira utilizar algo mais preciso, é recomendado utilizar os modelos `lg` (large) ou ainda carregar outros dicionários de vetores, como o word2vec, fastText, etc. O `gensim` é uma biblioteca que auxilia o uso de tais dicionários.

In [ ]:
w1 = nlp('gato')
w2 = nlp('cachorro')
w3 = nlp('presidente')

In [ ]:
w1.similarity(w2)

In [ ]:
w2.similarity(w3)

Similaridade entre textos:

In [ ]:
doc_review = nlp(review)
doc_wiki   = nlp(content)

doc_review.similarity(doc_wiki)